# BIP0039 生成确定性密钥的助记码

<https://en.bitcoin.it/wiki/BIP_0039>

**BTC私钥** 实际上就是一个32字节的随机数组，而 **公钥** 是通过将椭圆曲线的 **生成元点** 自加而得到，自加的次数就是这个私钥 ，此时私钥被当作一个大整数。而 **BTC地址** 则是通过把公钥哈希而得到。程序逻辑如下：

例子来自 <https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses>

In [2]:
import random

# 设置随机数种子，以便结果可重复
random.seed(0)

# 随机生成一个私钥
private_key = random.randint(0, 2**256 - 1)

# 为了方便，我们直接使用一个固定的私钥
private_key = 0x18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725
print(f"private_key: 0x{private_key:064x}")

private_key: 0x18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725


In [3]:
# BTC所选择的椭圆曲线是secp256k1, 其参数如下
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0
b = 7
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8

# 椭圆曲线的方程是 y^2 = x^3 + ax + b

# 椭圆曲线上的加法如下，其中 None 表示无穷远点，即加法的单位元，即 0，p1和p2是椭圆曲线上的点
def secp256k1_add(p1, p2):
    if p1 is None:
        return p2
    if p2 is None:
        return p1
    if p1[0] == p2[0] and p1[1] != p2[1]:
        # 两个点互为相反数，即相加为0
        assert (p1[1] + p2[1] % p) == 0
        return None
    # s是斜率
    if p1[0] == p2[0]:
        s = (3 * p1[0] * p1[0] + a) * pow(2 * p1[1], -1, p)
    else:
        s = (p2[1] - p1[1]) * pow(p2[0] - p1[0], -1, p)
    x = (s * s - p1[0] - p2[0]) % p
    y = ( - (s * (x - p1[0]) + p1[1]) ) % p
    return (x, y)

# 私钥 -> 公钥 (倍点算法)
def priv_key_to_pub_key(priv_key):
    # 生成公钥
    pub_key = None
    base = (Gx, Gy)
    for i in range(256):
        if priv_key & (1 << i):
            pub_key = secp256k1_add(pub_key, base)
        base = secp256k1_add(base, base)
    return pub_key

# 生成公钥
pub_key = priv_key_to_pub_key(private_key)
print(f"pub_key: 0x{pub_key[0]:064x}, 0x{pub_key[1]:064x}")

# 压缩形式的公钥
if pub_key[1] & 1:
    compressed_pub_key = 0x03
else:
    compressed_pub_key = 0x02
compressed_pub_key = compressed_pub_key << 256 | pub_key[0]
print(f"compressed_pub_key: 0x{compressed_pub_key:066x}")


pub_key: 0x50863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352, 0x2cd470243453a299fa9e77237716103abc11a1df38855ed6f2ee187e9c582ba6
compressed_pub_key: 0x0250863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352


In [5]:
import hashlib

# 生成地址
hashed_pub_key = hashlib.sha256(compressed_pub_key.to_bytes(33, 'big')).digest()
print(f"sha256ed_pub_key: 0x{int.from_bytes(hashed_pub_key, 'big'):064x}")

ripemd160_hasher = hashlib.new('ripemd160')
ripemd160_hasher.update(hashed_pub_key)
ripemd160_pub_key = ripemd160_hasher.digest()
print(f"ripemd160ed_pub_key: 0x{int.from_bytes(ripemd160_pub_key, 'big'):040x}")

ripemd160_pub_key_with_version = int.from_bytes(ripemd160_pub_key, 'big').to_bytes(21, 'big') # 添加版本号
print(f"ripemd160_pub_key_with_version: 0x{int.from_bytes(ripemd160_pub_key_with_version, 'big'):042x}")

hashed_ripemd160_pub_key_with_version = hashlib.sha256(ripemd160_pub_key_with_version).digest()
print(f"sha256ed_ripemd160_pub_key_with_version: 0x{int.from_bytes(hashed_ripemd160_pub_key_with_version, 'big'):064x}")

hashed_twice_ripemd160_pub_key_with_version = hashlib.sha256(hashed_ripemd160_pub_key_with_version).digest()
print(f"sha256ed_twice_ripemd160_pub_key_with_version: 0x{int.from_bytes(hashed_twice_ripemd160_pub_key_with_version, 'big'):064x}")

checksum = hashed_twice_ripemd160_pub_key_with_version[:4]
print(f"checksum: 0x{int.from_bytes(checksum, 'big'):08x}")

address = ripemd160_pub_key_with_version + checksum
print(f"address: 0x{int.from_bytes(address, 'big'):050x}")

# base58编码
import base58
address = base58.b58encode(address)
print(f"base58 address: {address.decode('utf-8')}")
assert address.decode('utf-8') == "1PMycacnJaSqwwJqjawXBErnLsZ7RkXUAs"

sha256ed_pub_key: 0x0b7c28c9b7290c98d7438e70b3d3f7c848fbd7d1dc194ff83f4f7cc9b1378e98
ripemd160ed_pub_key: 0xf54a5851e9372b87810a8e60cdd2e7cfd80b6e31
ripemd160_pub_key_with_version: 0x00f54a5851e9372b87810a8e60cdd2e7cfd80b6e31
sha256ed_ripemd160_pub_key_with_version: 0xad3c854da227c7e99c4abfad4ea41d71311160df2e415e713318c70d67c6b41c
sha256ed_twice_ripemd160_pub_key_with_version: 0xc7f18fe8fcbed6396741e58ad259b5cb16b7fd7f041904147ba1dcffabf747fd
checksum: 0xc7f18fe8
address: 0x00f54a5851e9372b87810a8e60cdd2e7cfd80b6e31c7f18fe8
base58 address: 1PMycacnJaSqwwJqjawXBErnLsZ7RkXUAs


上面就是生成私钥，以及从私钥生成比特币地址的完整过程。为了防止币的丢失，就必须保证私钥被妥善保管。私钥的随机性，导致了私钥难于准确的被记录。BIP0039尝试解决这个问题。

BIP0039中，我们首先生成 **随机比特** ，然后根据随机比特生成 **助记词** ，然后根据助记词生成 **私钥**。我们只需要记住助记词，就能随时重新生成私钥，从而生成公钥以及地址。而助记词由若干单词组成，方便记忆或者记录。为了进一步方便记忆和记录，BIP0039对助记词的选择做了限制，并且还添加了额外的冗余防止记忆出错。

助记词词典由2048个英文单词组成。这个词典还有一些特殊性质，一切都是为了更容易记忆或者记录。

1. 只要前4个字母，就能区分所有单词。
2. 相似的词语不选用。
3. 单词列表是有序的。

完整的单词列表如下：

In [10]:
mnemonic_word_list = 'abandon ability able about above absent absorb abstract absurd abuse access accident account accuse achieve acid acoustic acquire across act action actor actress actual adapt add addict address adjust admit adult advance advice aerobic affair afford afraid again age agent agree ahead aim air airport aisle alarm album alcohol alert alien all alley allow almost alone alpha already also alter always amateur amazing among amount amused analyst anchor ancient anger angle angry animal ankle announce annual another answer antenna antique anxiety any apart apology appear apple approve april arch arctic area arena argue arm armed armor army around arrange arrest arrive arrow art artefact artist artwork ask aspect assault asset assist assume asthma athlete atom attack attend attitude attract auction audit august aunt author auto autumn average avocado avoid awake aware away awesome awful awkward axis baby bachelor bacon badge bag balance balcony ball bamboo banana banner bar barely bargain barrel base basic basket battle beach bean beauty because become beef before begin behave behind believe below belt bench benefit best betray better between beyond bicycle bid bike bind biology bird birth bitter black blade blame blanket blast bleak bless blind blood blossom blouse blue blur blush board boat body boil bomb bone bonus book boost border boring borrow boss bottom bounce box boy bracket brain brand brass brave bread breeze brick bridge brief bright bring brisk broccoli broken bronze broom brother brown brush bubble buddy budget buffalo build bulb bulk bullet bundle bunker burden burger burst bus business busy butter buyer buzz cabbage cabin cable cactus cage cake call calm camera camp can canal cancel candy cannon canoe canvas canyon capable capital captain car carbon card cargo carpet carry cart case cash casino castle casual cat catalog catch category cattle caught cause caution cave ceiling celery cement census century cereal certain chair chalk champion change chaos chapter charge chase chat cheap check cheese chef cherry chest chicken chief child chimney choice choose chronic chuckle chunk churn cigar cinnamon circle citizen city civil claim clap clarify claw clay clean clerk clever click client cliff climb clinic clip clock clog close cloth cloud clown club clump cluster clutch coach coast coconut code coffee coil coin collect color column combine come comfort comic common company concert conduct confirm congress connect consider control convince cook cool copper copy coral core corn correct cost cotton couch country couple course cousin cover coyote crack cradle craft cram crane crash crater crawl crazy cream credit creek crew cricket crime crisp critic crop cross crouch crowd crucial cruel cruise crumble crunch crush cry crystal cube culture cup cupboard curious current curtain curve cushion custom cute cycle dad damage damp dance danger daring dash daughter dawn day deal debate debris decade december decide decline decorate decrease deer defense define defy degree delay deliver demand demise denial dentist deny depart depend deposit depth deputy derive describe desert design desk despair destroy detail detect develop device devote diagram dial diamond diary dice diesel diet differ digital dignity dilemma dinner dinosaur direct dirt disagree discover disease dish dismiss disorder display distance divert divide divorce dizzy doctor document dog doll dolphin domain donate donkey donor door dose double dove draft dragon drama drastic draw dream dress drift drill drink drip drive drop drum dry duck dumb dune during dust dutch duty dwarf dynamic eager eagle early earn earth easily east easy echo ecology economy edge edit educate effort egg eight either elbow elder electric elegant element elephant elevator elite else embark embody embrace emerge emotion employ empower empty enable enact end endless endorse enemy energy enforce engage engine enhance enjoy enlist enough enrich enroll ensure enter entire entry envelope episode equal equip era erase erode erosion error erupt escape essay essence estate eternal ethics evidence evil evoke evolve exact example excess exchange excite exclude excuse execute exercise exhaust exhibit exile exist exit exotic expand expect expire explain expose express extend extra eye eyebrow fabric face faculty fade faint faith fall false fame family famous fan fancy fantasy farm fashion fat fatal father fatigue fault favorite feature february federal fee feed feel female fence festival fetch fever few fiber fiction field figure file film filter final find fine finger finish fire firm first fiscal fish fit fitness fix flag flame flash flat flavor flee flight flip float flock floor flower fluid flush fly foam focus fog foil fold follow food foot force forest forget fork fortune forum forward fossil foster found fox fragile frame frequent fresh friend fringe frog front frost frown frozen fruit fuel fun funny furnace fury future gadget gain galaxy gallery game gap garage garbage garden garlic garment gas gasp gate gather gauge gaze general genius genre gentle genuine gesture ghost giant gift giggle ginger giraffe girl give glad glance glare glass glide glimpse globe gloom glory glove glow glue goat goddess gold good goose gorilla gospel gossip govern gown grab grace grain grant grape grass gravity great green grid grief grit grocery group grow grunt guard guess guide guilt guitar gun gym habit hair half hammer hamster hand happy harbor hard harsh harvest hat have hawk hazard head health heart heavy hedgehog height hello helmet help hen hero hidden high hill hint hip hire history hobby hockey hold hole holiday hollow home honey hood hope horn horror horse hospital host hotel hour hover hub huge human humble humor hundred hungry hunt hurdle hurry hurt husband hybrid ice icon idea identify idle ignore ill illegal illness image imitate immense immune impact impose improve impulse inch include income increase index indicate indoor industry infant inflict inform inhale inherit initial inject injury inmate inner innocent input inquiry insane insect inside inspire install intact interest into invest invite involve iron island isolate issue item ivory jacket jaguar jar jazz jealous jeans jelly jewel job join joke journey joy judge juice jump jungle junior junk just kangaroo keen keep ketchup key kick kid kidney kind kingdom kiss kit kitchen kite kitten kiwi knee knife knock know lab label labor ladder lady lake lamp language laptop large later latin laugh laundry lava law lawn lawsuit layer lazy leader leaf learn leave lecture left leg legal legend leisure lemon lend length lens leopard lesson letter level liar liberty library license life lift light like limb limit link lion liquid list little live lizard load loan lobster local lock logic lonely long loop lottery loud lounge love loyal lucky luggage lumber lunar lunch luxury lyrics machine mad magic magnet maid mail main major make mammal man manage mandate mango mansion manual maple marble march margin marine market marriage mask mass master match material math matrix matter maximum maze meadow mean measure meat mechanic medal media melody melt member memory mention menu mercy merge merit merry mesh message metal method middle midnight milk million mimic mind minimum minor minute miracle mirror misery miss mistake mix mixed mixture mobile model modify mom moment monitor monkey monster month moon moral more morning mosquito mother motion motor mountain mouse move movie much muffin mule multiply muscle museum mushroom music must mutual myself mystery myth naive name napkin narrow nasty nation nature near neck need negative neglect neither nephew nerve nest net network neutral never news next nice night noble noise nominee noodle normal north nose notable note nothing notice novel now nuclear number nurse nut oak obey object oblige obscure observe obtain obvious occur ocean october odor off offer office often oil okay old olive olympic omit once one onion online only open opera opinion oppose option orange orbit orchard order ordinary organ orient original orphan ostrich other outdoor outer output outside oval oven over own owner oxygen oyster ozone pact paddle page pair palace palm panda panel panic panther paper parade parent park parrot party pass patch path patient patrol pattern pause pave payment peace peanut pear peasant pelican pen penalty pencil people pepper perfect permit person pet phone photo phrase physical piano picnic picture piece pig pigeon pill pilot pink pioneer pipe pistol pitch pizza place planet plastic plate play please pledge pluck plug plunge poem poet point polar pole police pond pony pool popular portion position possible post potato pottery poverty powder power practice praise predict prefer prepare present pretty prevent price pride primary print priority prison private prize problem process produce profit program project promote proof property prosper protect proud provide public pudding pull pulp pulse pumpkin punch pupil puppy purchase purity purpose purse push put puzzle pyramid quality quantum quarter question quick quit quiz quote rabbit raccoon race rack radar radio rail rain raise rally ramp ranch random range rapid rare rate rather raven raw razor ready real reason rebel rebuild recall receive recipe record recycle reduce reflect reform refuse region regret regular reject relax release relief rely remain remember remind remove render renew rent reopen repair repeat replace report require rescue resemble resist resource response result retire retreat return reunion reveal review reward rhythm rib ribbon rice rich ride ridge rifle right rigid ring riot ripple risk ritual rival river road roast robot robust rocket romance roof rookie room rose rotate rough round route royal rubber rude rug rule run runway rural sad saddle sadness safe sail salad salmon salon salt salute same sample sand satisfy satoshi sauce sausage save say scale scan scare scatter scene scheme school science scissors scorpion scout scrap screen script scrub sea search season seat second secret section security seed seek segment select sell seminar senior sense sentence series service session settle setup seven shadow shaft shallow share shed shell sheriff shield shift shine ship shiver shock shoe shoot shop short shoulder shove shrimp shrug shuffle shy sibling sick side siege sight sign silent silk silly silver similar simple since sing siren sister situate six size skate sketch ski skill skin skirt skull slab slam sleep slender slice slide slight slim slogan slot slow slush small smart smile smoke smooth snack snake snap sniff snow soap soccer social sock soda soft solar soldier solid solution solve someone song soon sorry sort soul sound soup source south space spare spatial spawn speak special speed spell spend sphere spice spider spike spin spirit split spoil sponsor spoon sport spot spray spread spring spy square squeeze squirrel stable stadium staff stage stairs stamp stand start state stay steak steel stem step stereo stick still sting stock stomach stone stool story stove strategy street strike strong struggle student stuff stumble style subject submit subway success such sudden suffer sugar suggest suit summer sun sunny sunset super supply supreme sure surface surge surprise surround survey suspect sustain swallow swamp swap swarm swear sweet swift swim swing switch sword symbol symptom syrup system table tackle tag tail talent talk tank tape target task taste tattoo taxi teach team tell ten tenant tennis tent term test text thank that theme then theory there they thing this thought three thrive throw thumb thunder ticket tide tiger tilt timber time tiny tip tired tissue title toast tobacco today toddler toe together toilet token tomato tomorrow tone tongue tonight tool tooth top topic topple torch tornado tortoise toss total tourist toward tower town toy track trade traffic tragic train transfer trap trash travel tray treat tree trend trial tribe trick trigger trim trip trophy trouble truck true truly trumpet trust truth try tube tuition tumble tuna tunnel turkey turn turtle twelve twenty twice twin twist two type typical ugly umbrella unable unaware uncle uncover under undo unfair unfold unhappy uniform unique unit universe unknown unlock until unusual unveil update upgrade uphold upon upper upset urban urge usage use used useful useless usual utility vacant vacuum vague valid valley valve van vanish vapor various vast vault vehicle velvet vendor venture venue verb verify version very vessel veteran viable vibrant vicious victory video view village vintage violin virtual virus visa visit visual vital vivid vocal voice void volcano volume vote voyage wage wagon wait walk wall walnut want warfare warm warrior wash wasp waste water wave way wealth weapon wear weasel weather web wedding weekend weird welcome west wet whale what wheat wheel when where whip whisper wide width wife wild will win window wine wing wink winner winter wire wisdom wise wish witness wolf woman wonder wood wool word work world worry worth wrap wreck wrestle wrist write wrong yard year yellow you young youth zebra zero zone zoo'
mnemonic_word_list = mnemonic_word_list.split(' ')

assert len(mnemonic_word_list) == 2048

# 检查前4位是否能唯一确定一个助记词
def check_mnemonic_word_list(mnemonic_word_list):
    mnemonic_word_list = list(mnemonic_word_list)
    mnemonic_word_list = [word[:4] for word in mnemonic_word_list]
    mnemonic_word_list = set(mnemonic_word_list)
    assert len(mnemonic_word_list) == 2048
check_mnemonic_word_list(mnemonic_word_list)

print(f"mnemonic_word_list: {','.join(mnemonic_word_list[0:4])},...")

mnemonic_word_list: abandon,ability,able,about,...


助记词的个数可以是12个或者更多，我们以12个助记词为例来说明BIP0039。

12个助记词，每个助记词从2048个单词中随机挑选，说明我们至少需要 `12 * 11 = 132` 个随机比特。在BIP0039中，12个助记词我们使用4个比特的校验码，因此剩余的128个比特来自随机。示例如下：

测试例子来自 <https://github.com/trezor/python-mnemonic/blob/master/vectors.json>

In [25]:
random_bits = random.randint(0, 2**128 - 1)

# 为了方便，我们直接使用一个固定的随机数
random_bits = 0x9e885d952ad362caeb4efe34a8e91bd2

random_bytes = random_bits.to_bytes(16, 'big')
sha256ed_random_bytes = hashlib.sha256(random_bytes).digest()
sha256ed_random_bytes_as_int = int.from_bytes(sha256ed_random_bytes, 'big')
random_bits = (random_bits << 4) | (sha256ed_random_bytes_as_int >> (256 - 4))

mnemonic = []
for i in range(12):
    mnemonic.append(mnemonic_word_list[random_bits & 2047])
    random_bits = random_bits >> 11
mnemonic = mnemonic[::-1]
mnemonic = ' '.join(mnemonic)
print(f"mnemonic: {mnemonic}")

mnemonic: ozone drill grab fiber curtain grace pudding thank cruise elder eight picnic


In [28]:
import unicodedata

salt = "mnemonic" + "TREZOR"
seed = hashlib.pbkdf2_hmac('sha512', mnemonic.encode('utf-8'), salt.encode('utf-8'), 2048)
print(f"seed: 0x{int.from_bytes(seed, 'big'):0128x}")


seed: 0x274ddc525802f7c828d8ef7ddbcdc5304e87ac3535913611fbbfa986d0c9e5476c91689f9c8a54fd55bd38606aa6a8595ad213d4c9c9f9aca3fb217069a41028
